In [ ]:
# https://en.wikipedia.org/wiki/Big_Five_personality_traits
# https://ipip.ori.org/newBigFive5broadKey.htm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
file_to_open = pd.read_csv("Resources/data.csv", delimiter="\t")
data=file_to_open.copy()
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.isnull().values.any()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
print(f"Number of participants: {len(data)}")

In [ ]:
data.age.value_counts()

In [ ]:
# calculate maximum year we're looking for in the data
max_year = 2012-13
print(f"Youngest participient's date of birth: {max_year}")

In [ ]:
data= data.loc[data["age"]<2000]
data.head()

In [ ]:
data["age"] = [2012-x if x >1960 else x for x in data["age"]]
data= data.loc[data["age"]<=120]
data.head()

In [ ]:
data.age.sort_values(ascending=False) 

In [ ]:
data.describe()

In [ ]:
# data.dtypes

In [ ]:
data.corr()

In [ ]:
data.country.value_counts()

In [ ]:
data.country.value_counts().mean()

In [ ]:
countries = pd.DataFrame(data["country"].value_counts())
c_125= countries.loc[countries["country"]>125]
plt.bar(c_125.index,c_125["country"])
plt.show()

In [ ]:
c_125.nunique()

In [ ]:
# Recheck how many cluster we should have as maximum
from sklearn.preprocessing import MinMaxScaler

df = data.drop("country", axis=1)
trim_df = df.drop(df.columns[:6], axis=1)
columns = list(trim_df.columns)
scaler = MinMaxScaler(feature_range=(0,1))
scale_df = scaler.fit_transform(trim_df)
scale_df = pd.DataFrame(scale_df, columns=columns)
scale_df.head()

In [ ]:
#Elbow Method 
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,15))
visualizer.fit(df)
visualizer.poof()

In [ ]:
# cluster analysis- Aysin

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)

In [ ]:
kmeans.fit(scale_df)

In [ ]:
predicted_clusters = kmeans.predict(scale_df)

In [ ]:
centers = kmeans.cluster_centers_
labels = kmeans.labels_
labels

In [ ]:
len(labels)

In [ ]:
centers

In [ ]:
# scale_df = scale_df.add(labels)
scale_df['cluster'] = labels.tolist()
scale_df

In [ ]:
# analysis of the clusters- Michael

In [ ]:
# analysis of the questions- Stephen
# mean response of the questions

In [ ]:
# demographic analysis-James
#gender, age

In [ ]:
#demographic analysis -Hai
# country, handedness,engnat

In [ ]:
# country_df = scale_df
# country_df['country'] = data['country']
# country_df

In [ ]:
age_df = scale_df
age_df['age'] = data['age']
age_df

In [ ]:
#
#cluster0_df = age_df[age_df['cluster']==0]
# cluster0x_df = cluster0_df.loc[:, 'E1':'O10']
# cluster0y_df =  cluster0_df.loc[:, 'age']
#cluster0y_df.head()
#cluster0x_df.head()
#cluster0_df.head()

In [ ]:
cluster0_df = age_df[age_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_age=cluster0_corr.loc['age',:]

cluster1_df = age_df[age_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_age=cluster1_corr.loc['age',:]

cluster2_df = age_df[age_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_age=cluster2_corr.loc['age',:]

cluster3_df = age_df[age_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_age=cluster3_corr.loc['age',:]

cluster4_df = age_df[age_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_age=cluster4_corr.loc['age',:]

combine_age_df = pd.DataFrame({'cluster0':cluster0_age,'cluster1':cluster1_age,'cluster2':cluster2_age,'cluster3':cluster3_age,'cluster4':cluster4_age})
combine_age_df.head()

In [ ]:
combine_age_df = pd.DataFrame({'cluster0':cluster0_age,'cluster1':cluster1_age,'cluster2':cluster2_age,'cluster3':cluster3_age,'cluster4':cluster4_age})
combine_age_df.head()


In [ ]:
engnat_df = scale_df
engnat_df['engnat'] = data['engnat']
engnat_df=engnat_df.drop(['age'],axis =1)
engnat_df

In [ ]:
cluster0_df = engnat_df[engnat_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_engnat=cluster0_corr.loc['engnat',:]

cluster1_df = engnat_df[engnat_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_engnat=cluster1_corr.loc['engnat',:]

cluster2_df = engnat_df[engnat_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_engnat=cluster2_corr.loc['engnat',:]

cluster3_df = engnat_df[engnat_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_engnat=cluster3_corr.loc['engnat',:]

cluster4_df = engnat_df[engnat_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_engnat=cluster4_corr.loc['engnat',:]

combine_engnat_df = pd.DataFrame({'cluster0':cluster0_engnat,'cluster1':cluster1_engnat,'cluster2':cluster2_engnat,'cluster3':cluster3_engnat,'cluster4':cluster4_engnat})
combine_engnat_df.head()


In [ ]:

race_df = scale_df
race_df['race'] = data['race']
race_df=race_df.drop(['age','engnat'],axis =1)
race_df

In [ ]:
cluster0_df = race_df[race_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_race=cluster0_corr.loc['race',:]

cluster1_df = race_df[race_df['race']==1]
cluster1_corr = cluster1_df.corr()
cluster1_race=cluster1_corr.loc['race',:]

cluster2_df = race_df[race_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_race=cluster2_corr.loc['race',:]

cluster3_df = race_df[race_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_race=cluster3_corr.loc['race',:]

cluster4_df = race_df[race_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_race=cluster4_corr.loc['race',:]

combine_race_df = pd.DataFrame({'cluster0':cluster0_race,'cluster1':cluster1_race,'cluster2':cluster2_race,'cluster3':cluster3_race,'cluster4':cluster4_race})
combine_race_df.head()

In [ ]:
gender_df = scale_df
gender_df['gender'] = data['gender']
gender_df=gender_df.drop(['age','engnat','race'],axis =1)
gender_df

In [ ]:
cluster0_df = gender_df[gender_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_gender=cluster0_corr.loc['gender',:]

cluster1_df = gender_df[gender_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_gender=cluster1_corr.loc['gender',:]

cluster2_df = gender_df[gender_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_gender=cluster2_corr.loc['gender',:]

cluster3_df = gender_df[gender_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_gender=cluster3_corr.loc['gender',:]

cluster4_df = gender_df[gender_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_gender=cluster4_corr.loc['gender',:]
#cluster1_gender.head()
combine_gender_df = pd.DataFrame({'cluster0':cluster0_gender,'cluster1':cluster1_gender,'cluster2':cluster2_gender,'cluster3':cluster3_gender,'cluster4':cluster4_gender})
combine_gender_df.head()

In [ ]:
hand_df = scale_df
hand_df['hand'] = data['hand']
hand_df=hand_df.drop(['age','engnat','race','gender'],axis =1)
hand_df

In [ ]:
cluster0_df = hand_df[hand_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_hand=cluster0_corr.loc['hand',:]

cluster1_df = hand_df[hand_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_hand=cluster1_corr.loc['hand',:]

cluster2_df = hand_df[hand_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_hand=cluster2_corr.loc['hand',:]

cluster3_df = hand_df[hand_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_hand=cluster3_corr.loc['hand',:]

cluster4_df = hand_df[hand_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_hand=cluster4_corr.loc['hand',:]

combine_hand_df = pd.DataFrame({'cluster0':cluster0_hand,'cluster1':cluster1_hand,'cluster2':cluster2_hand,'cluster3':cluster3_hand,'cluster4':cluster4_hand})
combine_hand_df.head()